# Image Processing SS 16 - Assignment - 10

### Deadline is 6.7.2016 at 16:00 o'clock

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import scipy.io.wavfile
from skimage.data import astronaut
from skimage.color import rgb2gray

from __future__ import division
import random
try:
    from StringIO import StringIO as BytesIO
except ImportError:
    from io import BytesIO
    
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
    
    
from numpy.fft import ifft2, fft2
from scipy.fftpack import dct
from PIL import Image
import itertools
import IPython
import os
import zipfile
import base64
from scipy import signal

pylab.rcParams['figure.figsize'] = (10, 10)   # This makes the plot bigger

In [ ]:
kernel_size = 21
def degradation(img):
    """The corruption process"""
    kernel = np.diag(np.ones((kernel_size,))) / kernel_size
    
    return np.real(ifft2(fft2(img) * fft2(kernel, img.shape)))

def degradation_change_one_pixel(img):
    corrupted = img.copy()
    corrupted[150, 100] = 0
    return corrupted

def degradation_add_noise(img):
    print(img.max())
    return np.clip(img + np.random.normal(0, 0.01, img.shape), 0, 1)

In [ ]:
f = urllib2.urlopen("https://images-na.ssl-images-amazon.com/images/I/5191B5zqR8L._SX359_BO1,204,203,200_.jpg")
img = rgb2gray(plt.imread(f, format='jpeg') / 255.)
print(img.shape)
img = img[:256, :256]

In [ ]:
degradated_img = degradation(img)
degradeted_img_pixel = degradation_change_one_pixel(degradated_img)
degradeted_img_noise = degradation_add_noise(degradated_img)

plt.subplot(221)
plt.title("Original image")
plt.imshow(img, cmap='gray')
plt.subplot(222)
plt.title("Degradeted image")
plt.imshow(degradated_img, cmap='gray')
plt.subplot(223)
plt.title("Degradeted image + one pixel set zero")
plt.imshow(degradeted_img_pixel, cmap='gray')
plt.subplot(224)
plt.title("Degradeted image + small gaussian noise")
plt.imshow(degradeted_img_noise, cmap='gray')

# Exercise 1 - Point Spread Function - 1 Point

In [ ]:
# Estimate the Point Spread Function
psf = np.zeros((kernel_size, kernel_size))
# your code here
plt.imshow(psf, cmap='gray')
plt.show()
#psf.sum()


# Exercise 2 - Implement Inverse Filtering + Butterworth filter - 5 Points

Simply divide the image by the estimated PSF in fourier space.

In [ ]:
def plot_reconstruction(reconstruction):
    plt.subplot(221)
    plt.title("Original image")
    plt.imshow(img, cmap='gray')
    plt.subplot(222)
    plt.title("Reconsructed degradeted image")
    plt.imshow(reconstruction(degradated_img), cmap='gray')
    plt.subplot(223)
    plt.title("Reconsructed degradeted image + one pixel set zero")
    plt.imshow(reconstruction(degradeted_img_pixel), cmap='gray')
    plt.subplot(224)
    plt.title("Reconsructed degradeted image + small gaussian noise")
    plt.imshow(reconstruction(degradeted_img_noise), cmap='gray')

In [ ]:
def flip_fft2(x):
    """Flip the result of the fft2 function, so that the deep frequiencies are in the center.
       If you flip a ft specturm two times it is in the original form again.
    """
    n = x.shape[0]
    assert n == x.shape[1]
    nh = n//2
    x[:nh, :nh] = np.fliplr(np.flipud(x[:nh, :nh]))
    x[nh:, :nh] = x[nh:, :nh].T
    x[:nh, nh:] = x[:nh, nh:].T
    x[nh:, nh:] = np.fliplr(np.flipud(x[nh:, nh:]))
    return x


plt.subplot(131)
plt.title("Spectrum from numpy")
plt.imshow(np.log(np.abs(fft2(img))), cmap='gray')

plt.subplot(132)
plt.title("Flipped spectrum")
flip_ft_img = flip_fft2(fft2(img))
plt.imshow(np.log(np.abs(flip_ft_img)), cmap='gray')
plt.subplot(133)
plt.title("Double Flip")
double_flip_ft_img = flip_fft2(flip_fft2(fft2(img)))
plt.imshow(ifft2(double_flip_ft_img).real, cmap='gray')
plt.show()

In [ ]:
def butterworth(order, cut_off, n=512):
    """
    Args:
        order (int): Order of the butterworth filter
        cut_off (float): Between 0-1 the cutoff frequiency. One is the nyquist frequency
        n (int): Number of sample to return
    Returns:
        w (ndarray): Frequiencies from 0 to 1. 
        h (ndarray): Coefficents how the frequiencies are scaled.
    """
    cut_off = 0.125
    b, a = signal.butter(order, cut_off, 'low')
    w, h = signal.freqz(b, a, worN=n)
    return w / np.pi, h

In [ ]:
# example butterworth filter
order = 1
cut_off = 0.125   # <- you can play at the cutoff frequiency.
w, h = butterworth(order, cut_off)
plt.figure(figsize=(10, 4))
plt.plot(w, 20*np.log10(np.abs(h)))
plt.xscale('log')
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [Nyquist frequency]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(cut_off, color='green') # cutoff frequency
plt.show()

In [ ]:
def butterworth_lowpass_filter(img_ft):
    """Filters the fourier transformation of the image with the butterworth filter"""
    # your code here
    return img_ft


def inverse_filtering(img):
    """Simple inverse filtering with a butterworth filter"""
    # your code here, select suitable parameters for the butterworth filter
    return img

plot_reconstruction(inverse_filtering)

# Exercise 3 - Wiener Filter - 4 Points

In [ ]:
# estimate the snr of the image
snr = 0

def wiener_filter(img):
    # your code here
    return img



In [ ]:
plot_reconstruction(wiener_filter)